In [1]:
import redis
import os
import matplotlib.pyplot as plt
plt.ion()

# Open Redis
REDIS_IP_ADDRESS = os.getenv("REDIS_IP_ADDRESS", "localhost")
REDIS_PORT = int(os.getenv("REDIS_PORT", "6379"))
# Open connection to redis here and store the client as a property of this object
    
redis_client = redis.Redis(host=REDIS_IP_ADDRESS, port=REDIS_PORT, db=0)

In [2]:
# Set the key
key = 'sim_ts'
#key = 'web_ts'

In [3]:
# Get Range (zpopmax and min are destructive they pull the data out of the set)
max_d = redis_client.zpopmax(key)
#print(max)

min_d = redis_client.zpopmin(key)
#print(min)
# Put them back
max_member = max_d[0][0]
max_score  = max_d[0][1]
redis_client.zadd(key, {max_member: max_score})

min_member = min_d[0][0]
min_score  = min_d[0][1]
redis_client.zadd(key, {min_member: min_score})

print("Min",min_score,"Max",max_score)
# Calculate the time interval
dif_score = max_score - min_score
# In milliseconds
print("Milliseconds", dif_score)
# Seconds
seconds = dif_score/1000.0
print("Seconds", seconds)
# Minutes
minutes = seconds/60
print("Minutes", minutes)
# Hours
hours = minutes/60
print("Hours", hours)

Min 1623440832070.0 Max 1623677465004.0
Milliseconds 236632934.0
Seconds 236632.934
Minutes 3943.8822333333333
Hours 65.73137055555556


In [4]:
# One hour is how many ms
ms_hr = 60 * 60 * 1000
print(ms_hr)

3600000


In [5]:
import math
# Calc Interactions Per Hour
hour_blocks = math.floor(hours + 1)
interactions_per_hr = []
for h in range(hour_blocks):
    selection_min = min_score + (h * ms_hr)
    selection_max = ms_hr + selection_min

    results = redis_client.zrangebyscore(key, selection_min, selection_max)
    
    interactions_per_hr.append(len(results))
    

In [6]:
# Plot
#%matplotlib widget
#import matplotlib.pyplot as plt
%matplotlib widget
x = list(range(len(interactions_per_hr)))
plt.plot(x,interactions_per_hr)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Areas with a 120 second gap seperate "sessions"
ms_gap = 120 * 1000
# Looking for gaps of this size using a similar technique as above.
steps = int(dif_score/ms_gap)
print(steps)
gaps_location = []
all_cnt = []
gap_counter_list = []
gap_counter = 0
session_counter_list = []
session_counter = 0
session_state = 0
session_length_list = []
session_length_counter = 0
for g in range(steps):
    selection_min = min_score + (g * ms_gap)
    selection_max = ms_gap + selection_min

    results = redis_client.zrangebyscore(key, selection_min, selection_max)
    all_cnt.append(len(results))
    
    if len(results) == 0:
        gaps_location.append([selection_min])
        if gap_counter < 10:
            gap_counter += 1
        # A sesssion may have ended
        if session_state == 1:
            session_counter += 1
            session_state = 0
            session_length_list.append(session_length_counter)
            session_length_counter = 0
    else:
        gap_counter = 0
        # A sesssion has started
        session_state = 1
        session_length_counter += 1
        
    gap_counter_list.append(gap_counter)
    session_counter_list.append(session_counter)

print(len(gaps_location))

gap_cnt = len(gaps_location)

print("Percent gaps", float(gap_cnt)/float(steps))

x = list(range(len(all_cnt)))
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

print("Number of sessions",max(session_counter_list))

%matplotlib widget
plt.plot(x,all_cnt)
plt.plot(x,gap_counter_list)
plt.plot(x,session_counter_list)
plt.show()


1971
1872
Percent gaps 0.9497716894977168
Number of sessions 74


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:



%matplotlib widget
x = session_length_list
num_bins = 5
n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …